In [1]:
#G. MIGNOT - Jan 13th 2018
#Data Wrangling - Json ex

#imports
import pandas as pd
import json
from pandas.io.json import json_normalize

#loading the json data into a dataframe df
df = pd.read_json('data/world_bank_projects.json')

df.info()
#500 rows. I can use country name as 500 entries are in it (no null entries)

#Does the amount of country names match the amount of country codes ? It should !
if df['countryname'].nunique()==df['countrycode'].nunique():
    print('Amount of Country Names matches amount of Country Codes :)!\n There are '+str(df['countrycode'].nunique())+' in total.')
else :print('Mismatch. There may be errors in either one of these columns.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

In [2]:
#1 - Find the 10 countries with most projects
#Each row of df is a project --> I can use a counter and then most_common(10) to quickly display the answer

from collections import Counter
most_pjcts = Counter(df['countryname']).most_common(10)
print(most_pjcts)
#Here is the output : 
# [("People's Republic of China", 19), ('Republic of Indonesia', 19), ('Socialist Republic of Vietnam', 17), ('Republic of India', 16), ('Republic of Yemen', 13), ('Kingdom of Morocco', 12), ('Nepal', 12), ("People's Republic of Bangladesh", 12), ('Republic of Mozambique', 11), ('Africa', 11)]


[("People's Republic of China", 19), ('Republic of Indonesia', 19), ('Socialist Republic of Vietnam', 17), ('Republic of India', 16), ('Republic of Yemen', 13), ('Kingdom of Morocco', 12), ('Nepal', 12), ("People's Republic of Bangladesh", 12), ('Republic of Mozambique', 11), ('Africa', 11)]


In [3]:
#2 - Find the top 10 major project themes (using column 'mjtheme_namecode')

#Exploring the data, I can see that there are multiple codes per project --> Linked to multiple themes !
#See here below for the first row, I have codes 8 and 11. Using get because it is a dict
example_codes= [item.get('code') for item in df['mjtheme_namecode'][0]]
example_names= [item.get('name') for item in df['mjtheme_namecode'][0]]
print(example_codes, example_names)
#OUTPUT for first run is : ['8', '11'] ['Human development', '']
#No value for code 11 in this case !!

#let's create a dict with each code and associated name
themes = {}
#Function to add new elements to dict AND complete if name not already written
def addtodict(my_dict,keys_list,values_list):
    for key in keys_list:
            if (key not in my_dict.keys()) or (key in my_dict.keys() and my_dict[key]==''):
                my_dict[key]=values_list[keys_list.index(key)]
    return my_dict

#let's create a list of the codes at the same time. We'll use this later to identify the most common ones
list_codes = []

for elmt in df['mjtheme_namecode']:
    codes = [i.get('code') for i in elmt]
    names = [i.get('name') for i in elmt]
    codes = [int(x) for x in codes] #codes must be integer to easily sort the table later based on this if needed.
    addtodict(themes,codes,names)
    list_codes.append(codes)

#List of all themes is in themes variable (dict. Now convert it to dataframe, but needs to go through series)
themes_df = pd.Series(themes,name='Name')
themes_df.reset_index()
themes_df = pd.DataFrame(themes_df)
themes_df
#themes_df is a dataframe with index = code !!

#Fill missing names

#list of the names, with index = code-1 (so to access code 2 --> list_themes[1])
list_themes = list(themes_df['Name'])

#Function to fill values = '' with appropriate name

def complete_dict(lod,values_list):#lod stands for list of dictionaries
    for dic in lod:
        if dic['name']=='':
            dic['name']= values_list[int(dic['code'])-1]

for elmt in df['mjtheme_namecode']:
    complete_dict(elmt,list_themes)

#Let's perform the same test as previously to see if the values has been filled properly:
example_codes= [item.get('code') for item in df['mjtheme_namecode'][0]]
example_names= [item.get('name') for item in df['mjtheme_namecode'][0]]
print(example_codes, example_names)
#OUTPUT : ['8', '11'] ['Human development', 'Environment and natural resources management']
#OK the table is filled :)!
df['mjtheme_namecode']

['8', '11'] ['Human development', '']
['8', '11'] ['Human development', 'Environment and natural resources management']


0      [{'code': '8', 'name': 'Human development'}, {...
1      [{'code': '1', 'name': 'Economic management'},...
2      [{'code': '5', 'name': 'Trade and integration'...
3      [{'code': '7', 'name': 'Social dev/gender/incl...
4      [{'code': '5', 'name': 'Trade and integration'...
5      [{'code': '6', 'name': 'Social protection and ...
6      [{'code': '2', 'name': 'Public sector governan...
7      [{'code': '11', 'name': 'Environment and natur...
8      [{'code': '10', 'name': 'Rural development'}, ...
9      [{'code': '2', 'name': 'Public sector governan...
10     [{'code': '10', 'name': 'Rural development'}, ...
11     [{'code': '10', 'name': 'Rural development'}, ...
12     [{'code': '4', 'name': 'Financial and private ...
13     [{'code': '5', 'name': 'Trade and integration'...
14     [{'code': '6', 'name': 'Social protection and ...
15     [{'code': '10', 'name': 'Rural development'}, ...
16     [{'code': '10', 'name': 'Rural development'}, ...
17     [{'code': '8', 'name': '

In [4]:
#Let's now give the 10 major project themes
#We'll use our previously build list of codes and count occurences.

#let's flatten the list of list we have with all codes:
list_codes_flat = [code for sublist in list_codes for code in sublist]

#We can now count occurences:
count_codes = Counter(list_codes_flat).most_common(10)
top_ten_themes = pd.DataFrame(count_codes,columns=['Code','Count'])
top_ten_themes.sort_values(by='Count',ascending=False,inplace=True)

top_ten_themes['Theme']=top_ten_themes.apply(lambda row: list_themes[int(row['Code'])-1] ,axis=1)
#Let's now have a look at the table of the 10 major themes in all the projects:
top_ten_themes

,Code,Count,Theme
0,11,250,Environment and natural resources management
1,10,216,Rural development
2,8,210,Human development
3,2,199,Public sector governance
4,6,168,Social protection and risk management
5,4,146,Financial and private sector development
6,7,130,Social dev/gender/inclusion
7,5,77,Trade and integration
8,9,50,Urban development
9,1,38,Economic management
